In [ ]:
!gdown 1uK0mhDl8tcYCSrEKkvqaoTZVjz_xMNp-

Downloading...
From: https://drive.google.com/uc?id=1uK0mhDl8tcYCSrEKkvqaoTZVjz_xMNp-
To: /content/679071.zip
100% 107M/107M [00:00<00:00, 167MB/s] 


In [ ]:
!unzip 679071.zip

Archive:  679071.zip
  inflating: content/679071.csv      


In [ ]:
import numpy as np
from keras import backend as K
from keras.models import Model
from keras.layers.core import Activation
from keras.layers import Input, Embedding, Flatten, dot
from keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files

In [ ]:
import pandas as pd
df=pd.read_csv('content/679071.csv')

<ipython-input-7-26eb05763e74>:2: DtypeWarning: Columns (0,1,2,3,4,5,6,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('content/679071.csv')


In [ ]:
y=df['Media']


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df,y,test_size=0.2)

In [ ]:
x_train.head()

,Unnamed: 0,DateTime,TweetId,Text,Username,Language,Hashtags,ReplyCount,RetweetCount,LikeCount,QuoteCount,Media,date,processed_text,LemmatizeText
511136,577477,2023-02-08 19:14:49+00:00,1623399971838992384,Friends of Woodberry Down will be hosting a ba...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254336,287714,2023-02-12 04:47:09+00:00,1624631167289856001,@me_think_free @APhilosophae And earthquake in...,InaUM3,en,NaN,1.0,0.0,1,0,NaN,2023-02-12,think free earthquake turkey syria,think free earthquake turkey syria
108829,121784,2023-02-06 16:37:15+00:00,1622635543564546049,"Nations rush to aid Turkiye, Syria as second e...",MiddleEastMnt,en,NaN,1.0,10.0,17,0,NaN,2023-02-06,nations rush aid turkiye syria second earthqua...,nation rush aid turkiye syria second earthquak...
614584,on the day. All funds raised will be donated ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
588054,on Friday 10th at 3.30pm. We are asking for d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
x_train.to_csv('content/train/train.csv')

In [ ]:
x_test.to_csv('content/test/test.csv')

In [ ]:

!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 12.9 MB/s eta 0:00:00


In [ ]:
import re
import logging
import numpy as np
import pandas as pd
import multiprocessing

from re import sub
from time import time 
from unidecode import unidecode
from gensim.models import Word2Vec
from collections import defaultdict
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.models.phrases import Phrases, Phraser

logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [ ]:
file_cleaned = df.Text.dropna().drop_duplicates().reset_index(drop=True)

In [ ]:
file_cleaned

0         89-member medical team has been dispatched by ...
1         EARTHQUAKE RELIEF IN TURKEY | Aerial Recovery\...
2         The Mozaik Foundation have launched an Emergen...
3         This call to contribute to earthquake relief i...
4         In compliance with the royal orders of Sultan ...
                                ...                        
667459    According to biblical prophecy, earthquakes ar...
667460    Are the earthquakes causing the shift in the t...
667461    Do you have a story to tell? Are you KP citize...
667462    #NowPlaying The Earthquake (Stefano Carparelli...
667463    When earth its horrid jaws half open shows, /\...
Name: Text, Length: 667464, dtype: object

In [ ]:
from unidecode import unidecode

def text_to_word_list(text, remove_polish_letters):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = remove_polish_letters(text)
    text = str(text)
    text = text.lower()

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ! ", text)
    text = sub(r"\?", " ? ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " : ", text)
    text = sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

df.Preprocess_Text = file_cleaned.apply(lambda x: text_to_word_list(x, unidecode))

<ipython-input-25-6c247c0efa1e>:25: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.Preprocess_Text = file_cleaned.apply(lambda x: text_to_word_list(x, unidecode))


In [ ]:
df.Preprocess_Text

0         [89, member, medical, team, has, been, dispatc...
1         [earthquake, relief, in, turkey, aerial, recov...
2         [the, mozaik, foundation, have, launched, an, ...
3         [this, call, to, contribute, to, earthquake, r...
4         [in, compliance, with, the, royal, orders, of,...
                                ...                        
667459    [according, to, biblical, prophecy, earthquake...
667460    [are, the, earthquakes, causing, the, shift, i...
667461    [do, you, have, a, story, to, tell, ?, are, yo...
667462    [nowplaying, the, earthquake, stefano, carpare...
667463    [when, earth, its, horrid, jaws, half, open, s...
Name: Text, Length: 667464, dtype: object

In [ ]:
df.Preprocess_Text


0         [89, member, medical, team, has, been, dispatc...
1         [earthquake, relief, in, turkey, aerial, recov...
2         [the, mozaik, foundation, have, launched, an, ...
3         [this, call, to, contribute, to, earthquake, r...
4         [in, compliance, with, the, royal, orders, of,...
                                ...                        
667459    [according, to, biblical, prophecy, earthquake...
667460    [are, the, earthquakes, causing, the, shift, i...
667461    [do, you, have, a, story, to, tell, ?, are, yo...
667462    [nowplaying, the, earthquake, stefano, carpare...
667463    [when, earth, its, horrid, jaws, half, open, s...
Name: Text, Length: 667464, dtype: object

In [ ]:

file_model = df.Preprocess_Text.copy

In [ ]:
file_model

<bound method NDFrame.copy of 0         [89, member, medical, team, has, been, dispatc...
1         [earthquake, relief, in, turkey, aerial, recov...
2         [the, mozaik, foundation, have, launched, an, ...
3         [this, call, to, contribute, to, earthquake, r...
4         [in, compliance, with, the, royal, orders, of,...
                                ...                        
667459    [according, to, biblical, prophecy, earthquake...
667460    [are, the, earthquakes, causing, the, shift, i...
667461    [do, you, have, a, story, to, tell, ?, are, yo...
667462    [nowplaying, the, earthquake, stefano, carpare...
667463    [when, earth, its, horrid, jaws, half, open, s...
Name: Text, Length: 667464, dtype: object>

In [ ]:
sent = [row for row in df.Preprocess_Text]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

['earthquake',
 'relief',
 'in',
 'turkey',
 'aerial',
 'recovery',
 'for',
 'those',
 'looking',
 'to',
 'help',
 'or',
 'donate',
 'i_m',
 'on',
 'the',
 'board',
 'of',
 'aerial',
 'recovery',
 'and',
 'we',
 'already',
 'have',
 'a',
 'team',
 'in',
 'route',
 'to',
 'help',
 'with',
 'humanitarian_aid',
 'https',
 '//t',
 'co/n0setno4zw']

In [ ]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

Time to build vocab: 0.39 mins


In [ ]:

start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

<ipython-input-44-9cdc5012bacd>:7: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


Time to train the model: 24.3 mins


In [ ]:
w2v_model.save("word2vec.model")

In [ ]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
w2v_model.save("/content/drive/MyDrive/sentimentAnalysis/word2vec.model")

In [ ]:
df.Preprocess_Text

0         [89, member, medical, team, has, been, dispatc...
1         [earthquake, relief, in, turkey, aerial, recov...
2         [the, mozaik, foundation, have, launched, an, ...
3         [this, call, to, contribute, to, earthquake, r...
4         [in, compliance, with, the, royal, orders, of,...
                                ...                        
667459    [according, to, biblical, prophecy, earthquake...
667460    [are, the, earthquakes, causing, the, shift, i...
667461    [do, you, have, a, story, to, tell, ?, are, yo...
667462    [nowplaying, the, earthquake, stefano, carpare...
667463    [when, earth, its, horrid, jaws, half, open, s...
Name: Text, Length: 667464, dtype: object

In [ ]:
file_export = df.Preprocess_Text.apply(lambda x: ' '.join(bigram[x]))

In [ ]:
file_export.to_csv('cleaned_dataset.csv', index=False)

In [170]:
file_export.to_csv('/content/drive/MyDrive/sentimentAnalysis/cleaned_dataset.csv', index=False)

In [ ]:
file_export.head()

0    89_member medical_team has_been dispatched by ...
1    earthquake relief in turkey aerial recovery fo...
2    the mozaik_foundation have launched_an emergen...
3    this call to contribute to earthquake relief i...
4    in compliance_with the royal_orders of sultan_...
Name: Text, dtype: object

In [ ]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

In [ ]:
word_vectors = Word2Vec.load("word2vec.model").wv


In [ ]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors.astype('double'))


In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)


[('booty_shake', 0.9920558333396912),
 ('by_aleeza', 0.9881709218025208),
 ('bournemouth_collection', 0.9871681928634644),
 ('atop_mauna', 0.9854265451431274),
 ('abbas_bga', 0.9843932390213013),
 ('co/waav1cfb4i', 0.9843568205833435),
 ('camera_perched', 0.9834827184677124),
 ('war_turkey/', 0.9826160073280334),
 ('merciful_intervention', 0.982100248336792),
 ('russia/_ukraine', 0.9820183515548706)]

In [ ]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]


In [ ]:
words = pd.DataFrame(word_vectors.index_to_key)
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])


In [ ]:
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value


In [ ]:
words.head(10)


,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,the,"[0.08377729, 0.05638026, -0.12238357, -0.09374...",0,-1,0.978203,-0.978203
1,earthquake,"[0.057303764, 0.094342776, -0.09663061, -0.066...",0,-1,0.980576,-0.980576
2,//t,"[-0.0050678886, 0.049971312, -0.0770737, 0.023...",1,1,0.987389,0.987389
3,https,"[0.00018257686, 0.04906287, -0.071466066, 0.01...",1,1,0.985411,0.985411
4,in,"[0.09040143, 0.063673146, -0.13173272, -0.0660...",0,-1,0.968891,-0.968891
5,turkey,"[0.010150742, 0.056697067, -0.09225638, -0.055...",0,-1,0.962042,-0.962042
6,and,"[0.07645951, 0.07227616, -0.10814511, -0.13355...",0,-1,0.972723,-0.972723
7,to,"[0.06698111, 0.05488286, -0.078557126, -0.0644...",0,-1,0.982932,-0.982932
8,of,"[0.05381538, 0.053523116, -0.10498798, -0.0646...",0,-1,0.980073,-0.980073
9,syria,"[-0.013126305, 0.08672077, -0.09425365, -0.071...",0,-1,0.964116,-0.964116


In [ ]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [169]:
words[['words', 'sentiment_coeff']].to_csv('/content/drive/MyDrive/sentimentAnalysis/sentiment_dictionary.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score


In [ ]:
final_file = pd.read_csv('cleaned_dataset.csv')


In [ ]:
sentiment_map = pd.read_csv('sentiment_dictionary.csv')
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))


In [ ]:
file_weighting = final_file.copy()


In [ ]:
file_weighting.Text

0         89_member medical_team has_been dispatched by ...
1         earthquake relief in turkey aerial recovery fo...
2         the mozaik_foundation have launched_an emergen...
3         this call to contribute to earthquake relief i...
4         in compliance_with the royal_orders of sultan_...
                                ...                        
667459    according to biblical_prophecy earthquakes are...
667460    are the earthquakes causing the shift in the t...
667461    do you have a story to tell ? are you kp citiz...
667462    nowplaying the earthquake stefano_carparelli b...
667463    when earth its horrid jaws half open shows / m...
Name: Text, Length: 667464, dtype: object

In [ ]:
sentiment_map

,words,sentiment_coeff
0,the,-0.978203
1,earthquake,-0.980576
2,//t,0.987389
3,https,0.985411
4,in,-0.968891
...,...,...
130615,yeah_idk,-1.118582
130616,ankhae,1.116550
130617,ozanbilaloglu,1.208359
130618,king_visiting,1.117188


In [ ]:
sentiment_dict

{'the': -0.9782029104247226,
 'earthquake': -0.9805764616248396,
 '//t': 0.9873886669638092,
 'https': 0.9854114058703368,
 'in': -0.9688911574963032,
 'turkey': -0.9620417053561192,
 'and': -0.9727229604937572,
 'to': -0.9829324901251042,
 'of': -0.980072983133568,
 'syria': -0.9641160927858008,
 'a': -0.9894811940362768,
 'for': -0.9907573870750797,
 'is': -1.0041023583487043,
 'earthquakes': -0.976751850194586,
 's': -1.0058001222057016,
 'people': -0.957148416191949,
 'are': -0.975402834368356,
 'that': -0.9870310013896904,
 'from': -0.9799587766927292,
 '!': -1.0120058220815535,
 'on': -1.001682049330755,
 'this': -1.0033012777444754,
 'after': -0.9614394783743432,
 'by': -0.9799734048387476,
 'with': -0.9773475644148648,
 'it': -1.012116153463628,
 'help': -0.9712266349826708,
 '?': -1.0455369428921375,
 'turkiye': -0.9433473659070772,
 'amp': -0.9860098789858596,
 'as': -0.9755371632284636,
 'we': -0.9869456419673728,
 'all': -0.9874723400096218,
 'i': -1.0048908504674756,
 'vic

In [ ]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf.fit(file_weighting.Text)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(norm=None, tokenizer=<function <lambda> at 0x7f0eb5074a60>)

In [ ]:

features = pd.Series(tfidf.get_feature_names_out())
transformed = tfidf.transform(file_weighting.Text)


In [ ]:
transformed

<667464x831499 sparse matrix of type '<class 'numpy.float64'>'
	with 14164016 stored elements in Compressed Sparse Row format>

In [ ]:
tfidf.get_feature_names_out()

array(['!', '!_0xed67922e36ed2422d391306f6f6ecc19d58eaa4f', '!_2023/2/8',
       ..., 'zzzaikar', 'zzzzz', 'zzzzzz'], dtype=object)

In [ ]:
file_weighting.Text

0         89_member medical_team has_been dispatched by ...
1         earthquake relief in turkey aerial recovery fo...
2         the mozaik_foundation have launched_an emergen...
3         this call to contribute to earthquake relief i...
4         in compliance_with the royal_orders of sultan_...
                                ...                        
667459    according to biblical_prophecy earthquakes are...
667460    are the earthquakes causing the shift in the t...
667461    do you have a story to tell ? are you kp citiz...
667462    nowplaying the earthquake stefano_carparelli b...
667463    when earth its horrid jaws half open shows / m...
Name: Text, Length: 667464, dtype: object

In [ ]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    print(vector_coo)

    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

In [ ]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer

    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.Text.split()))


In [ ]:
replaced_tfidf_scores = file_weighting.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)#this step takes around 3-4 minutes minutes to calculate


In [ ]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out


In [ ]:
replaced_closeness_scores = file_weighting.Text.apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))


In [ ]:
len(replaced_closeness_scores)

667464

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, file_weighting.Text,np.zeros(667464)]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence','sentiment']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]


In [ ]:
replacement_df.head()

,sentiment_coeff,tfidf_scores,sentence,sentiment,sentiment_rate,prediction
0,"[1.083278598569799, -0.9697309385804362, -0.96...","[11.11540536742055, 8.530709247024179, 5.14603...",89_member medical_team has_been dispatched by ...,0,-119.661876,0
1,"[-0.9805764616248396, -0.9843021256154544, -0....","[1.2062910820851462, 4.510408198354314, 3.3749...",earthquake relief in turkey aerial recovery fo...,0,-183.453875,0
2,"[-0.9782029104247226, -0.9917034716429152, -0....","[6.299007596345884, 10.627052599506618, 3.9252...",the mozaik_foundation have launched_an emergen...,0,-219.423624,0
3,"[-1.0033012777444754, -0.9978815738872362, -0....","[3.3210552052332716, 6.498087047496811, 5.7963...",this call to contribute to earthquake relief i...,0,-195.022992,0
4,"[-0.9688911574963032, -1.0300744553954169, -0....","[1.6874984482290247, 11.11540536742055, 4.7242...",in compliance_with the royal_orders of sultan_...,0,-12.322221,0


In [ ]:
replacement_df[["sentence","prediction"]].head(100)

,sentence,prediction
0,89_member medical_team has_been dispatched by ...,0
1,earthquake relief in turkey aerial recovery fo...,0
2,the mozaik_foundation have launched_an emergen...,0
3,this call to contribute to earthquake relief i...,0
4,in compliance_with the royal_orders of sultan_...,0
...,...,...
95,garudaaerospace drone manufacturer garuda_aero...,0
96,i have wanted to be a part of muratpak s proje...,0
97,this may be the top declaration of the growthf...,0
98,india_sends 5 flights with rescue_teams relief...,0


In [ ]:
replacement_df[replacement_df['prediction']==0].count()
#positive tweets

sentiment_coeff    608076
tfidf_scores       608076
sentence           608076
sentiment          608076
sentiment_rate     608076
prediction         608076
dtype: int64

In [ ]:
replacement_df[replacement_df['prediction']==1].count()
#negative tweets

sentiment_coeff    59388
tfidf_scores       59388
sentence           59388
sentiment          59388
sentiment_rate     59388
prediction         59388
dtype: int64

In [167]:
replacement_df.to_csv("/content/drive/MyDrive/sentimentAnalysis/predictedData.csv")




In [168]:
replacement_df.head()

,sentiment_coeff,tfidf_scores,sentence,sentiment,sentiment_rate,prediction
0,"[1.083278598569799, -0.9697309385804362, -0.96...","[11.11540536742055, 8.530709247024179, 5.14603...",89_member medical_team has_been dispatched by ...,0,-119.661876,0
1,"[-0.9805764616248396, -0.9843021256154544, -0....","[1.2062910820851462, 4.510408198354314, 3.3749...",earthquake relief in turkey aerial recovery fo...,0,-183.453875,0
2,"[-0.9782029104247226, -0.9917034716429152, -0....","[6.299007596345884, 10.627052599506618, 3.9252...",the mozaik_foundation have launched_an emergen...,0,-219.423624,0
3,"[-1.0033012777444754, -0.9978815738872362, -0....","[3.3210552052332716, 6.498087047496811, 5.7963...",this call to contribute to earthquake relief i...,0,-195.022992,0
4,"[-0.9688911574963032, -1.0300744553954169, -0....","[1.6874984482290247, 11.11540536742055, 4.7242...",in compliance_with the royal_orders of sultan_...,0,-12.322221,0


In [ ]:
predicted_classes = replacement_df.prediction
y_test = replacement_df.sentiment

conf_matrix = pd.DataFrame(confusion_matrix(replacement_df.sentiment, replacement_df.prediction))
print('Confusion Matrix')
display(conf_matrix)

test_scores = accuracy_score(y_test,predicted_classes), precision_score(y_test, predicted_classes), recall_score(y_test, predicted_classes), f1_score(y_test, predicted_classes)

print('\n \n Scores')
scores = pd.DataFrame(data=[test_scores])
scores.columns = ['accuracy', 'precision', 'recall', 'f1']
scores = scores.T
scores.columns = ['scores']
display(scores)

Confusion Matrix


,0,1
0,608076,59388
1,0,0



 
 Scores


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,scores
accuracy,0.911024
precision,0.000000
recall,0.000000
f1,0.000000


In [171]:
!pip install Pattern


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 KB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.4/348.4 KB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 19.5 MB/s eta 0:00:00
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332718 sha256=4c8cba8e

In [172]:
!pip install Pattern


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [173]:
from pattern.en import sentiment
print(sentiment("The course of true love never did run smooth."))


(0.4166666666666667, 0.5833333333333334)


In [174]:
replacement_df.patternPrediction = replacement_df.sentence.apply(lambda x:sentiment(x))

<ipython-input-174-30170a8ba37f>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  replacement_df.patternPrediction = replacement_df.sentence.apply(lambda x:sentiment(x))


In [214]:
a = replacement_df.patternPrediction

In [229]:
a.iloc[0][0]<0

False

In [234]:
positive = 0
negative = 0
for i in range(len(a)):
  b = a.iloc[i][0]
  if b < 0:
    negative = negative + 1
  else:
    positive = positive + 1

In [235]:
positive

520224

In [236]:
negative

147240

In [184]:

replacement_df.head(100)

,sentiment_coeff,tfidf_scores,sentence,sentiment,sentiment_rate,prediction
0,"[1.083278598569799, -0.9697309385804362, -0.96...","[11.11540536742055, 8.530709247024179, 5.14603...",89_member medical_team has_been dispatched by ...,0,-119.661876,0
1,"[-0.9805764616248396, -0.9843021256154544, -0....","[1.2062910820851462, 4.510408198354314, 3.3749...",earthquake relief in turkey aerial recovery fo...,0,-183.453875,0
2,"[-0.9782029104247226, -0.9917034716429152, -0....","[6.299007596345884, 10.627052599506618, 3.9252...",the mozaik_foundation have launched_an emergen...,0,-219.423624,0
3,"[-1.0033012777444754, -0.9978815738872362, -0....","[3.3210552052332716, 6.498087047496811, 5.7963...",this call to contribute to earthquake relief i...,0,-195.022992,0
4,"[-0.9688911574963032, -1.0300744553954169, -0....","[1.6874984482290247, 11.11540536742055, 4.7242...",in compliance_with the royal_orders of sultan_...,0,-12.322221,0
...,...,...,...,...,...,...
95,"[1.222085250440056, -0.9985622018235496, -1.03...","[12.619482764196823, 9.00407046196476, 10.8558...",garudaaerospace drone manufacturer garuda_aero...,0,-47.495666,0
96,"[-1.0048908504674756, -0.983859208201685, -0.9...","[3.8438303590057084, 3.925203921670832, 7.8473...",i have wanted to be a part of muratpak s proje...,0,-225.241415,0
97,"[-1.0033012777444754, -0.9809738677318072, -1....","[3.3210552052332716, 5.359077296414572, 4.4180...",this may be the top declaration of the growthf...,0,-261.982283,0
98,"[-0.9787251411510924, -0.957980883800028, -0.9...","[8.567697816393519, 4.3861124349066465, 8.1709...",india_sends 5 flights with rescue_teams relief...,0,-26.748552,0
